In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'/home/paladin/Downloads/Facial_Impression_Recognition_Calassification'

In [3]:
from cnnClassifier.utils import load_object
import keras

2023-08-01 10:36:31.925606: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-01 10:36:31.991754: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-01 10:36:31.993067: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-01 10:36:33.155945: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
model = load_object(path='artifacts/training/model.h5', h5=True)

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path    
    validation_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int


In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils import read_yaml, create_directories

In [7]:
class configurationManeger:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 secret_filepath = SECRET_FILE_PATH,                 
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath) 
        self.secret = read_yaml(secret_filepath)        
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])  

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model= Path('artifacts/training/model.h5'),           
            validation_data= Path('artifacts/data_transformation/validation/'),
            all_params = self.params,
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE
            )
        
        return eval_config

In [8]:
import sys
from cnnClassifier.exception import CustomException
from cnnClassifier.utils import save_json, load_object

In [9]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            # Dividing the pixels by 255 for normalization  => range(0,1)
            # Scaling the pixels value in range(-1,1) by subtracting 0.5 and multiply 2
            rescale= ((1./255) - 0.5)*2,            
            # if there was no validation set:
            # validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],            
            batch_size = self.config.params_batch_size,
            interpolation= 'bilinear'
        )

        datagenerator = keras.preprocessing.image.ImageDataGenerator(
                **datagenerator_kwargs
        )
        
        ## NOTE: subset is added when we use validation_split, where directory for both training and validation is the same.
        ## NOTE: shuffle= False for validation as we want to check the performance model using predict_generator 
        self.valid_generator = datagenerator.flow_from_directory(
            directory= self.config.validation_data,
            #subset= "validation",
            shuffle= False,            
            **dataflow_kwargs
            )
        
         
    def evaluation(self):
        model = load_object(path=self.config.path_of_model, h5=True)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {'loss': self.score[0], 'accuracy': self.score[1]}
        save_json(path='scores.json', data=scores)


In [10]:
try:
    config = configurationManeger()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise CustomException(e, sys)


Found 3589 images belonging to 7 classes.
113/113 [==============================] - 10s 87ms/step - loss: 0.9862 - accuracy: 0.6461
